# Model

In this notebook, clustering is not applied before training.

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('../data/boreholes_uscs_sptn.csv')
df

,Borehole ID,Latitude,Longitude,Elevation (ft),In-situ Test #,Test Elevation (ft),SPTN,Layer #,Layer Top Elevation (ft),Layer Bottom Elevation (ft),Soil Description,Lithology,USCS,Density / Consistency
0,21,47.568436,-122.407480,77.30,1,77.30,3.0,1,77.30,75.80,"Grayish brown, dry, very loose, well graded sa...",Fill,Fill,NaN
1,21,47.568436,-122.407480,77.30,2,75.80,12.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",Clay,CH,Stiff
2,21,47.568436,-122.407480,77.30,3,74.30,8.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",Clay,CH,Medium
3,21,47.568436,-122.407480,77.30,4,73.30,7.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",Clay,CH,Medium
4,21,47.568436,-122.407480,77.30,5,71.80,7.0,2,75.80,70.30,"Clay, medium gray, stiff, slightly moist, with...",Clay,CH,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19711,153205,47.437412,-122.242456,23.75,9,-8.75,44.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",Silty sand,SM,Dense
19712,153205,47.437412,-122.242456,23.75,10,-13.75,23.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",Silty sand,SM,Medium
19713,153205,47.437412,-122.242456,23.75,11,-18.75,43.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",Silty sand,SM,Dense
19714,153205,47.437412,-122.242456,23.75,12,-23.75,27.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",Silty sand,SM,Dense


In [ ]:
table_uscs = pd.read_csv('../data/file_2_reUSCS_sampling_with_elevation.csv')
table_uscs